# INSTALL AND IMPORT DEPENDENCIES

In [ ]:
# %pip install matplotlib
# %pip install easyocr
# %pip install imutils
# %pip install numpy
# %pip install pyqt5

In [ ]:
import cv2
import imutils
import easyocr
import numpy as np
from matplotlib import pyplot as plt

# (OPTIONAL) SELECT IMAGE TO PROCESS

In [ ]:
from PyQt5.QtWidgets import QApplication, QFileDialog

# Create a QApplication instance
app = QApplication([])

# Open the file dialog and get the selected file path
file_dialog = QFileDialog()
file_path, _ = file_dialog.getOpenFileName()

# Print the selected file path
print("Selected file:", file_path)

# Close the application
app.quit()

# READ AN IMAGE, GRAYSCALE AND BLUR

## IF NO IMAGE WAS SELECTED, THEN WRITE THE PATH OF THE IMAGE

In [ ]:
img = cv2.imread(file_path)

In [ ]:
print("IMAGEN A DETECTAR")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
# Convertir la imagen a espacio de color HSV
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Definir rango de colores rojos en HSV
lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])

# Crear una máscara que solo contiene los píxeles rojos
red_mask = cv2.inRange(hsv_img, lower_red, upper_red)

# Aplicar la máscara a la imagen original
img_without_red = cv2.bitwise_and(img, img, mask=~red_mask)

In [ ]:
# Convertir la imagen a escala de grises
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Aplicar umbralización para segmentar los píxeles rojos
_, red_thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

# Invertir la máscara de umbralización
red_thresh = cv2.bitwise_not(red_thresh)

# Aplicar la máscara a la imagen original
img_without_red = cv2.bitwise_and(img, img, mask=red_thresh)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
print("IMAGEN EN ESCALA DE GRISES")
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

In [ ]:
gray = cv2.blur(gray, (3, 3))

In [ ]:
print("IMAGEN EN ESCALA DE GRISES CON BLUR")
plt.imshow(gray, cmap="gray")

# APPLY FILTER AND FIND EDGES FOR LOCALIZATION

In [ ]:
# Aplicar filtrado Gaussiano para suavizar la imagen
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# Aplicar filtrado bilateral para reducir el ruido manteniendo los bordes
bilateral = cv2.bilateralFilter(blur, 11, 17, 17)

# Aplicar detección de bordes con Canny
edged = cv2.Canny(bilateral, 30, 200)

In [ ]:
print("IMAGEN CON BORDES DETECTADOS")
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

# FIND CONTOURS AND APPLY MASK

In [ ]:
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

In [ ]:
# Después de encontrar los contornos, filtrar aquellos que no tienen la forma deseada y el tamaño adecuado
filtered_contours = []
# min_area = 1000  # área mínima esperada para la placa del carro
# max_area = 5000  # área máxima esperada para la placa del carro
for contour in contours:
    # Calcular el área del contorno
    area = cv2.contourArea(contour)
    # Calcular el perímetro del contorno
    perimeter = cv2.arcLength(contour, True)
    # Aproximar el contorno a un polígono
    approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
    # Filtrar los contornos que tienen 4 lados, un área dentro del rango deseado y una relación de aspecto cercana a 1

    filtered_contours.append(contour)

In [ ]:
if filtered_contours:
    location = filtered_contours[0]
    print("POSIBLE UBICACION DE LA PLACA\n", location)

    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0, 255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)

    print("IMAGEN CON LA PLACA DETECTADA")
    plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))
else:
    print("No se encontraron contornos que parezcan ser una placa del carro.")

In [ ]:
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
cropped = gray[topx : bottomx + 1, topy : bottomy + 1]

In [ ]:
print("IMAGEN DE LA PLACA RECORTADA")
plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))

In [ ]:
kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

# Aplicar el filtro de enfoque a la imagen
sharpened = cv2.filter2D(cropped, -1, kernel)

# Guardar la imagen mejorada en la variable cropped
cropped = sharpened

In [ ]:
print("IMAGEN DE LA PLACA RECORTADA")
plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))

# USE EASY OCR TO READ TEXT

In [ ]:
reader = easyocr.Reader(["es"])
result = reader.readtext(cropped)

In [ ]:
print("TEXTO DETECTADO EN LA PLACA")
print(result)

In [ ]:
# Iterar sobre los resultados detectados
for detection in result:
    # Obtener el texto detectado y convertirlo a mayúsculas
    text = detection[-2].upper()

    # Crear una imagen en blanco del mismo tamaño que la original
    plate_image = np.zeros_like(img)

    # Dibujar un rectángulo en la imagen de la placa
    plate_image = cv2.rectangle(
        plate_image, tuple(approx[0][0]), tuple(approx[2][0]), (255, 255, 255), -1
    )

    # Escribir el texto de la placa en la imagen de la placa
    font_scale = 2
    font_thickness = 3
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    text_position = (
        (plate_image.shape[1] - text_size[0]) // 2,
        (plate_image.shape[0] + text_size[1]) // 2,
    )
    plate_image = cv2.putText(
        plate_image,
        text,
        text_position,
        font,
        font_scale,
        (0, 255, 0),
        font_thickness,
        cv2.LINE_AA,
    )

    # Mostrar la imagen con la placa simulada
    print("IMAGEN CON LA PLACA SIMULADA")
    plt.imshow(cv2.cvtColor(plate_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()